In [ ]:
import os
import cv2
import PIL
import glob
import time
import imageio
import zipfile
import numpy as np
from PIL import Image
import tensorflow as tf
from IPython import display
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading Zip File
zip_path = r'C:\Users\anshu\OneDrive\Desktop\dataset.zip'
extract_path = r'C:\Users\anshu\OneDrive\Desktop\extracted_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Defining Function & Loading Images
def load_images(image_folder, image_files):
    images=[]
    for file_name in image_files:
        if file_name.endswith(('.jpg','.jpeg')):
            img_path=os.path.join(image_folder, file_name)
            img= Image.open(img_path).convert('RGB')
            images.append(img)
    return images    

image_folder = extract_path
image_files = [file for file in os.listdir(image_folder) if file.lower().endswith(('jpg', 'jpeg'))]
images = load_images(image_folder, image_files)   

# Histogram Equalization
def clahe_equalization(img):
    img_lab = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2Lab)
    l, a, b = cv2.split(img_lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    img_lab = cv2.merge((l_clahe, a, b))
    img_equalized = cv2.cvtColor(img_lab, cv2.COLOR_Lab2RGB)
    return Image.fromarray(img_equalized)

equalized_images = [clahe_equalization(img) for img in images]

# Resizing & Normalizing
def process(equalized_images, size=(150, 212)):
    processed_images = []
    for img in equalized_images:
        img_resized = img.resize(size)
        if img_resized.mode != 'RGB':
            img_resized = img_resized.convert('RGB')
        img_array = np.array(img_resized) / 127.5 - 1.0
        processed_images.append(img_array)
    return np.array(processed_images)

resized_images = process(equalized_images)

# Augmentation
data_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

aug_images = []
for img in resized_images:
    img = np.expand_dims(img, axis=0)
    aug_iter = data_generator.flow(img)
    for _ in range(2):  # Number of augmentations per image
        aug_image = next(aug_iter)
        aug_images.append(aug_image[0])

# Training Set allocation
train_images = np.array([np.array(img) for img in aug_images])
BUFFER_SIZE = 20
BATCH_SIZE = 10
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Defining Generator Model
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(53*38*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((53, 38, 256)))
    assert model.output_shape == (None, 53, 38, 256)
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 106, 76, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 212, 152, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 212, 152, 3)
    return model

generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

# Convert the EagerTensor to a NumPy array before using astype
generated_image_np = generated_image[0].numpy()
generated_image_np = (generated_image_np * 127.5 + 127.5).astype(np.uint8)

# Defining Discriminator Model
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[212, 152, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Defining Discriminator Loss
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Defining Generator Loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Adding Chekpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Setting Noise & number of EPOCHS
EPOCHS = 300
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# Feedback Mechanism (Real and Fake Image differentiation)
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Generating and Saving Images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, :].numpy() * 127.5 + 127.5).astype(np.uint8))
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Training the Model
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)

        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)

train(train_dataset, EPOCHS)